## DISCOVERY EDA ADVANCED
- Revisar fecha de inicio y fin de cada serie. Revisar largo de cada serie. Hay series de distinto largo?
- Revisar cantidad de nulos. Ej día domingo. Que no hay ventas. Rellenar esos datos con ceros
- Revisar qué series tienen más volumen. Qué productos se venden más

In [3]:
import os
# set path root of repo
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('/')[:-1]
root_path = '/'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

root path:  /Users/joseortega/Documents/GitHub/forecasting-m5-dataset


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 1. Read raw files

In [5]:
folder_data_raw_dtype = 'data/data_input_dtype/'

df_calender = pd.read_pickle(folder_data_raw_dtype + 'calendar.pkl')
df_prices = pd.read_pickle(folder_data_raw_dtype + 'sell_prices.pkl')
df_sales = pd.read_pickle(folder_data_raw_dtype + 'sales_train_evaluation.pkl')
df_sample_output = pd.read_pickle(folder_data_raw_dtype + 'sample_submission.pkl')

In [6]:
df_sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,2.0,4.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,1.0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,4.0,0.0,1.0,3.0,0.0,2.0,6.0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0


In [7]:
df_sales.shape

(30490, 1947)

In [14]:
# cantidad única de series
df_sales['id'].unique().shape

(30490,)

In [25]:
# si en total hay 30490 series y la cantidad de observaciones son hasta el dia 1941.
# Se puede obtener la cantidad teórica de filas que debería haber si no existieran los nulos
shape_teorico = 30490 * 1941
shape_teorico

59181090

### 2. Transformar dataframe con las ventas a formato "id", "date", "value"

In [12]:
df_sales_long = df_sales.melt(
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
    var_name='d',
    value_name='y'
)

In [15]:
df_sales_long.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,y
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0


In [22]:
df_sales_long.tail(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d,y
59181087,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,2.0
59181088,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,0.0
59181089,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1941,1.0


In [16]:
df_sales_long.shape

(59181090, 8)

In [27]:
# revisar si todas las series parten desde la misma fecha (que la cantidad de filas coincida con el teórico)
shape_teorico == df_sales_long.shape[0]

True

### 2.b crear variable auxiliar que transformar el día (d_11) a integer (11)

In [30]:
df_sales_long['d_integer'] = df_sales_long['d'].str.extract(r'(\d+)', expand=False).astype(int)

In [31]:
df_sales_long.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d,y,d_integer
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,1
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,1


In [32]:
df_sales_long.tail(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d,y,d_integer
59181087,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,2.0,1941
59181088,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,0.0,1941
59181089,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1941,1.0,1941


### 3. Revisar fecha de inicio y fin de cada serie

In [121]:
# generar en dataframe donde se muestra información de cada una de las series
df_analize_each_serie = pd.DataFrame()

In [122]:
df_analize_each_serie = df_sales_long.groupby('id')['d_integer'].agg(['min', 'max']).reset_index()
df_analize_each_serie.rename(columns = {'min':'min_date', 'max':'max_date'}, inplace = True)

/var/folders/fx/0kx46vn52sg544t8x3s4hdl00000gn/T/ipykernel_31999/134287862.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_analize_each_serie = df_sales_long.groupby('id')['d_integer'].agg(['min', 'max']).reset_index()


In [123]:
# revisar si hay alguna serie que el día de INICIO sea distinto a 1
(df_start_end_date['min'] != 1).sum()

0

In [124]:
# revisar si hay alguna serie que el día de FIN sea distinto a 1941
(df_start_end_date['max'] != 1941).sum()

0

In [125]:
df_analize_each_serie.head()

,id,min_date,max_date
0,HOBBIES_1_001_CA_1_evaluation,1,1941
1,HOBBIES_1_002_CA_1_evaluation,1,1941
2,HOBBIES_1_003_CA_1_evaluation,1,1941
3,HOBBIES_1_004_CA_1_evaluation,1,1941
4,HOBBIES_1_005_CA_1_evaluation,1,1941


**POR LO TANTO, TODAS LAS SERIES TIENEN LAS MISMAS FECHA DE INICIO Y FIN. Además por el tamaño teórico NO hay nulos**

### 4. Revisar si hay nulos
Por el tamaño teórico de la data que coincide, se sabe que no existen nulos. De todas formas se calculan cuantas observaciones hay en cada serie

In [126]:
# calcular el tamaño de cada serie
df_aux_size_serie = df_sales_long.groupby('id').size().reset_index().rename(columns = {0:'size_serie'})

/var/folders/fx/0kx46vn52sg544t8x3s4hdl00000gn/T/ipykernel_31999/1543969028.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_aux_size_serie = df_sales_long.groupby('id').size().reset_index().rename(columns = {0:'size_serie'})


In [127]:
# hacer el join con el dataframe que tiene la info de cada serie
df_analize_each_serie = pd.merge(df_analize_each_serie, df_aux_size_serie, on = 'id', how='inner')

In [128]:
# revisar si hay alguna serie con largo distinto a 1941
(df_analize_each_serie['size_serie'] != 1941).sum()

0

In [129]:
df_analize_each_serie.head()

,id,min_date,max_date,size_serie
0,HOBBIES_1_001_CA_1_evaluation,1,1941,1941
1,HOBBIES_1_002_CA_1_evaluation,1,1941,1941
2,HOBBIES_1_003_CA_1_evaluation,1,1941,1941
3,HOBBIES_1_004_CA_1_evaluation,1,1941,1941
4,HOBBIES_1_005_CA_1_evaluation,1,1941,1941


**COMO SE MENCIONÓ, NO HAY VALORES NULOS. TODAS LAS SERIES TIENEN TODOS SUS DATOS**

### 5. Revisar qué series tienen más volumen (series de tiempo de ventas)

In [130]:
# calcular el volumen total de cada serie
df_aux_volumen_serie = df_sales_long.groupby('id')['y'].sum().reset_index().rename(columns = {'y':'volumen'})

# agregar columna donde se calcule el volumen porcentual con respecto al total de volumen
total_volumen = df_aux_volumen_serie['volumen'].sum()
df_aux_volumen_serie['volumen_percent'] = np.round( 100 * (df_aux_volumen_serie['volumen'] / total_volumen) , 5)

/var/folders/fx/0kx46vn52sg544t8x3s4hdl00000gn/T/ipykernel_31999/1407211194.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_aux_volumen_serie = df_sales_long.groupby('id')['y'].sum().reset_index().rename(columns = {'y':'volumen'})


In [131]:
# hacer el join con el dataframe que tiene la info de cada serie
df_analize_each_serie = pd.merge(df_analize_each_serie, df_aux_volumen_serie, on = 'id', how='inner')

In [132]:
# ordenar por volumen (el resto de columnas fecha de incio, fecha fin, tamaño serie son todos iguales)
df_analize_each_serie.sort_values(by = 'volumen', ascending = False, inplace = True)

In [138]:
# mostrar las 10 series con más volumen
df_analize_each_serie.head(10)

,id,min_date,max_date,size_serie,volumen,volumen_percent
8412,FOODS_3_090_CA_3_evaluation,1,1941,1941,253859.0,0.37931
18055,FOODS_3_586_TX_2_evaluation,1,1941,1941,195120.0,0.29154
21104,FOODS_3_586_TX_3_evaluation,1,1941,1941,151862.0,0.22691
8908,FOODS_3_586_CA_3_evaluation,1,1941,1941,136269.0,0.20361
2314,FOODS_3_090_CA_1_evaluation,1,1941,1941,128855.0,0.19253
29755,FOODS_3_090_WI_3_evaluation,1,1941,1941,123500.0,0.18453
17559,FOODS_3_090_TX_2_evaluation,1,1941,1941,121275.0,0.18120
20608,FOODS_3_090_TX_3_evaluation,1,1941,1941,116773.0,0.17448
17721,FOODS_3_252_TX_2_evaluation,1,1941,1941,115613.0,0.17274
15006,FOODS_3_586_TX_1_evaluation,1,1941,1941,114010.0,0.17035


In [141]:
# mostrar las 10 series con menos volumen
df_analize_each_serie.tail(10)

,id,min_date,max_date,size_serie,volumen,volumen_percent
6992,HOUSEHOLD_1_336_CA_3_evaluation,1,1941,1941,24.0,0.00004
10549,HOUSEHOLD_2_307_CA_4_evaluation,1,1941,1941,24.0,0.00004
17280,FOODS_2_209_TX_2_evaluation,1,1941,1941,24.0,0.00004
10458,HOUSEHOLD_2_216_CA_4_evaluation,1,1941,1941,23.0,0.00003
6682,HOUSEHOLD_1_020_CA_3_evaluation,1,1941,1941,23.0,0.00003
935,HOUSEHOLD_1_378_CA_1_evaluation,1,1941,1941,23.0,0.00003
20192,FOODS_2_071_TX_3_evaluation,1,1941,1941,18.0,0.00003
27606,HOBBIES_1_170_WI_3_evaluation,1,1941,1941,16.0,0.00002
26276,FOODS_2_057_WI_2_evaluation,1,1941,1941,16.0,0.00002
6048,FOODS_3_778_CA_2_evaluation,1,1941,1941,15.0,0.00002


In [142]:
# 10 series con más volumen, son el 2% de los datos
df_analize_each_serie.head(10)['volumen_percent'].sum()

2.1772